# Requirements:
- Define two new types: PlayingCard and CardDeck; # https://en.wikipedia.org/wiki/Standard_52-card_deck
- PlayingCard instances should support two instance attributes: suit and rank. The specified attributes should be
standardized to lowercase, but not otherwise validated. Equality and a good string representation should be supported.
- CardDeck will represent the traditional deck of 52 cards:
    - 4 suits ('spades', 'diamonds', 'clubs', 'hearts') * 13 ranks (2 through 10, inclusive, Ace, Jack, King, Queen)
- CardDeck should be a pythonic sequence supporting the following functionality:
  - membership testing with 'in' keyword (returns bool)
  - square bracket indexing with ints (PlayingCard instances should be returned)
  - slicing with slice objects (CardDeck should be returned)
  - iteration in for loops
  - concatenation through '+' operator, supported between two instances of CardDeck, or PlayingCard & CardDeck
  - concatenation/extension through '*' operator, supported between an instance of CardDeck and int
  - both of the above concatenation operations should be commutative
- If instantiated with no args, CardDeck should generate a full deck of 52 cards. If instantiated with a python list
of at least one PlayingCard, CardDeck should reflect only the list of cards passed to it at instantiation. If a python
list containing no valid PlayingCard instances, the default full deck of 52 cards should be generated.
- Finally, CardDeck should support a draw(n=1) method that randomly selects a card from a given deck and returns it
without replacement, i.e. the number of cards remaining in the deck decreases by 1
- If a user draws more than 1 card at a time, e.g. draw(n=3), a CardDeck type should be returned; if only one card
is drawn, a PlayingCard type should be returned.

In [2]:
class PlayingCard:

    def __init__(self, suits, rank):
        self.suit = suits.lower()
        self.rank = rank.lower()

    def __eq__(self, other):
        if not isinstance(other, type(self)):
            raise NotImplementedError(f"{other} must be of type self")
        return self.suit == other.suit and self.rank == other.rank

    def __repr__(self):
        return f"{type(self).__name__}(suit={self.suit!r}, rank={self.rank!r})"

In [3]:
# leaner object representation by limiting number of object in the console
import reprlib
deque_repr = reprlib.Repr()
deque_repr.maxother = 50
deque_repr.maxlist = 3   

import random

class CardDeck:

    SUITS = ('spades', 'diamonds', 'clubs', 'hearts')
    RANK = list('23456789') + ['10'] + list('ajkq')

    def __init__(self, cards=[]):
        self.cards=cards

    @property
    def cards(self):
        return self._cards

    @cards.setter
    def cards(self, value):
        self._cards = [card for card in value if isinstance(card, PlayingCard)]
        if not self._cards:
            self._cards = self._generate_full_deck()

    def __repr__(self):
        return f"{type(self).__name__}(cards={deque_repr.repr(self.cards)})"

    def __len__(self):
        return len(self.cards)

    def __getitem__(self, i):
        if type(i) == slice:
            return type(self)(self._cards[i]) # generate CardDeck Object
        return self._cards[i]

    def __contains__(self, value):
        if not isinstance(value, PlayingCard):
            raise NotImplementedError(f"{value} must be of type PlayingCard")
        return value in self._cards

    def __add__(self, other):
        if isinstance(other, type(self)):
            return type(self)(self._cards + other._cards)
        elif isinstance(other, PlayingCard):
            return type(self)(self._cards + [other])
        else:
            raise NotImplementedError(f"{other} must be of type self or PlayingCard") # return new CardDeck object

    def __radd__(self, other):
        return self.__add__(other)

    def __mul__(self, other):
        if not isinstance(other, int):
            raise NotImplementedError(f"{other} must be of type int")
        deque = []
        for card in self._cards:
            deque.extend([card for _ in range(other)])
        return type(self)(deque)

    def __rmul__(self, other):
        return self.__mul__(other)

    def __iter__(self):
        return (card for card in self._cards) # return a generator expression
            
    def _generate_full_deck(self):
        deck = [PlayingCard(suit, rank) for suit in self.SUITS for rank in self.RANK]
        return deck

    def _draw_one(self):
        return self._cards.pop(random.randint(0, len(self)))

    def draw(self, n=None):
        if not n:
            return self._draw_one()
        return type(self)([self._draw_one() for _ in range(n)])

In [4]:
cd = CardDeck()
cd

CardDeck(cards=[PlayingCard(suit='spades', rank='2'), PlayingCard(suit='spades', rank='3'), PlayingCard(suit='spades', rank='4'), ...])

In [5]:
cd2 = CardDeck(cards=[PlayingCard('diamonds', '2')])
cd3 = CardDeck(cards=["Andrew", PlayingCard('clubs', '7')])
cd4 = CardDeck(cards=["Andrew", "Lisa"])

In [6]:
len(cd), len(cd2), len(cd3), len(cd4)

(52, 1, 1, 52)

In [7]:
cd[1]

PlayingCard(suit='spades', rank='3')

In [8]:
cd[-10:]

CardDeck(cards=[PlayingCard(suit='hearts', rank='5'), PlayingCard(suit='hearts', rank='6'), PlayingCard(suit='hearts', rank='7'), ...])

In [9]:
PlayingCard('spADes', 'J') in cd

True

In [10]:
cd2 + cd3

CardDeck(cards=[PlayingCard(suit='diamonds', rank='2'), PlayingCard(suit='clubs', rank='7')])

In [11]:
cd2 + PlayingCard('spades', '6')

CardDeck(cards=[PlayingCard(suit='diamonds', rank='2'), PlayingCard(suit='spades', rank='6')])

In [12]:
PlayingCard('spades', '6') + cd3

CardDeck(cards=[PlayingCard(suit='clubs', rank='7'), PlayingCard(suit='spades', rank='6')])

In [13]:
cd2*3

CardDeck(cards=[PlayingCard(suit='diamonds', rank='2'), PlayingCard(suit='diamonds', rank='2'), PlayingCard(suit='diamonds', rank='2')])

In [14]:
3*cd2

CardDeck(cards=[PlayingCard(suit='diamonds', rank='2'), PlayingCard(suit='diamonds', rank='2'), PlayingCard(suit='diamonds', rank='2')])

In [15]:
for card in 3 * cd2:
    print(card)

PlayingCard(suit='diamonds', rank='2')
PlayingCard(suit='diamonds', rank='2')
PlayingCard(suit='diamonds', rank='2')


In [16]:
gen = iter(3 * cd2)
print(next(gen))
print(next(gen))

PlayingCard(suit='diamonds', rank='2')
PlayingCard(suit='diamonds', rank='2')


In [17]:
cd2

CardDeck(cards=[PlayingCard(suit='diamonds', rank='2')])

In [18]:
len(cd)

52

In [19]:
cd.draw()

PlayingCard(suit='clubs', rank='a')

In [20]:
len(cd)

51

In [21]:
print(len(cd))
print(cd.draw(4))
print(type(cd.draw(4)))
print(len(cd))

51
CardDeck(cards=[PlayingCard(suit='spades', rank='9'), PlayingCard(suit='spades', rank='6'), PlayingCard(suit='hearts', rank='j'), ...])
<class '__main__.CardDeck'>
43


In [22]:
PlayingCard('spADes', '6') == PlayingCard('spades', '6')

True